In [122]:
import openai
import numpy as np
import pandas as pd

In [124]:
openai.api_key = 'XXXXXXXX'

def gerar_descricao(c):
    return f"Cliente de porte {c['porte']}, ramo {c['ramo']}, localizado em {c['cidade']}, {c['uf']}"

In [126]:
df = pd.read_excel("clientes.xlsx")
df

,id,porte,ramo,cidade,uf,latitude,longitude
0,C001,médio,logística,Campinas,SP,-22.90,-47.06
1,C002,pequeno,varejo,Curitiba,PR,-25.43,-49.27
2,C003,grande,alimentício,Salvador,BA,-12.97,-38.51
3,C004,médio,logística,Campinas,SP,-22.90,-47.06
4,C005,pequeno,varejo,Curitiba,PR,-25.43,-49.27
5,C006,grande,alimentício,Salvador,BA,-12.97,-38.51
6,C007,médio,logística,Campinas,SP,-22.90,-47.06
7,C008,pequeno,varejo,Curitiba,PR,-25.43,-49.27
8,C009,grande,alimentício,Salvador,BA,-12.97,-38.51
9,C010,médio,logística,Campinas,SP,-22.90,-47.06


In [128]:
df_clusters = df[['porte', 'ramo', 'cidade', 'uf']].drop_duplicates().reset_index(drop=True)
df_clusters['cluster'] = df_clusters.index
df_clusters = df_clusters[['cluster', 'porte', 'ramo', 'cidade', 'uf']]
df_clusters

,cluster,porte,ramo,cidade,uf
0,0,médio,logística,Campinas,SP
1,1,pequeno,varejo,Curitiba,PR
2,2,grande,alimentício,Salvador,BA


In [130]:
df_clientes = df.merge(df_clusters, on=['porte', 'ramo', 'cidade', 'uf'], how='left')[['id', 'cluster', 'latitude', 'longitude']]
df_clientes

,id,cluster,latitude,longitude
0,C001,0,-22.90,-47.06
1,C002,1,-25.43,-49.27
2,C003,2,-12.97,-38.51
3,C004,0,-22.90,-47.06
4,C005,1,-25.43,-49.27
5,C006,2,-12.97,-38.51
6,C007,0,-22.90,-47.06
7,C008,1,-25.43,-49.27
8,C009,2,-12.97,-38.51
9,C010,0,-22.90,-47.06


In [132]:
df_clusters.to_excel('base_clusters.xlsx', index=False)
df_clientes.to_excel('base_clientes.xlsx', index=False)

In [134]:
clusters = df_clusters.to_dict(orient='records')

descricoes = [gerar_descricao(c) for c in clusters]
descricoes

['Cliente de porte médio, ramo logística, localizado em Campinas, SP',
 'Cliente de porte pequeno, ramo varejo, localizado em Curitiba, PR',
 'Cliente de porte grande, ramo alimentício, localizado em Salvador, BA']

In [136]:
response = openai.embeddings.create(
    model="text-embedding-ada-002",
    input=descricoes
)

vetores = np.array([d.embedding for d in response.data])
vetores

array([[-0.00236031, -0.01229498,  0.01010777, ..., -0.00878121,
        -0.01985316, -0.02792903],
       [-0.01621108, -0.01515808,  0.00503753, ..., -0.01136856,
        -0.02675414, -0.03647819],
       [-0.01515117, -0.02964644,  0.00021706, ..., -0.02202494,
        -0.01338025, -0.01250135]])

In [138]:
clusters = df_clusters["cluster"].values
np.savez("clusters_base.npz", vetores=vetores, clusters=clusters)